## CIA FACTBOOK COUNTRY COMPARISON USING SQLITE3

In [1]:
## Importing the Libraries

import sqlite3
import pandas as pd




In [2]:
## Creating the connection with the database and fetching the data
conn = sqlite3.connect("factbook.db")
database = "SELECT * FROM sqlite_master WHERE type='table';"
cursor = conn.cursor()
cursor.execute(database).fetchall()

[('table',
  'sqlite_sequence',
  'sqlite_sequence',
  3,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'facts',
  'facts',
  47,
  'CREATE TABLE "facts" ("id" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "code" varchar(255) NOT NULL, "name" varchar(255) NOT NULL, "area" integer, "area_land" integer, "area_water" integer, "population" integer, "population_growth" float, "birth_rate" float, "death_rate" float, "migration_rate" float)')]

In [3]:
## Reading the database using pandas libraries
pd.read_sql_query(database,conn)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY..."


In [4]:
## Calling the complete database
database1 = "SELECT * FROM facts LIMIT 10"
pd.read_sql_query(database1,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
5,6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
6,7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.00
7,8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.80
8,9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
9,10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


In [6]:
### Checking the Minimum and Maximum Population and population_growth
database2="SELECT MIN(population) AS Minimumm_population, MAX(population) AS Maximum_population, MIN(population_growth) AS Minimum_population_growth, MAX(population_growth) AS Maximum_population_growth FROM facts;"
pd.read_sql_query(database2,conn)

,Minimumm_population,Maximum_population,Minimum_population_growth,Maximum_population_growth
0,0,7256490011,0.0,4.02


### TASK 1- Which country has the highest population?

In [13]:
###Highest population of the country except world
database3="SELECT name, MAX(population) AS Maximum_population FROM facts WHERE name!='World';"
pd.read_sql_query(database3,conn)

,name,Maximum_population
0,China,1367485388


In [14]:
### Complete details about China
database4="SELECT *, MAX(population) AS Maximum_population FROM facts WHERE name!='World';"
pd.read_sql_query(database4,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,Maximum_population
0,37,ch,China,9596960,9326410,270550,1367485388,0.45,12.49,7.53,0.44,1367485388


### TASK 2-Which country has the least number of people?

In [15]:
### Lowest Population of the country in the World
database5="SELECT name, MIN(population) AS Minimum_population FROM facts;"
pd.read_sql_query(database5,conn)

,name,Minimum_population
0,Antarctica,0


In [17]:
### Complete Details about the Country
database6="SELECT *, Min(population) AS Minimum_population FROM facts WHERE name!='World';"
pd.read_sql_query(database6,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,Minimum_population
0,250,ay,Antarctica,None,280000,None,0,None,None,None,None,0


### TASK 3-Which country is witnessing the highest population growth?

In [18]:
database7="SELECT name, MAX(population_growth) AS Maximum_population_growth FROM facts;"
pd.read_sql_query(database7,conn)

,name,Maximum_population_growth
0,South Sudan,4.02


In [19]:
### Complete Details about the Country
database8="SELECT *, MAX(population_growth) AS Maximum_population_growth FROM facts WHERE name!='World';"
pd.read_sql_query(database8,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,Maximum_population_growth
0,162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47,4.02


### TASK 4-Which country has an extraordinary number for the population?

In [20]:
### Extraordinary population growth 
database9="SELECT * FROM facts WHERE name!='World' GROUP BY population_growth HAVING (SELECT MAX(population_growth) AS Maximum_population_growth FROM facts) ORDER BY population_growth DESC LIMIT 5 ;"
pd.read_sql_query(database9,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,162,od,South Sudan,644329.0,NaN,NaN,12042910,4.02,36.91,8.18,11.47
1,106,mi,Malawi,118484.0,94080.0,24404.0,17964697,3.32,41.56,8.41,0.00
2,29,by,Burundi,27830.0,25680.0,2150.0,10742276,3.28,42.01,9.27,0.00
3,128,ng,Niger,NaN,1266700.0,300.0,18045729,3.25,45.45,12.42,0.56
4,182,ug,Uganda,241038.0,197100.0,43938.0,37101745,3.24,43.79,10.69,0.74


### From the above table we can see that South Sudan and the other four nation has the extraordinary population growth

### TASk 5- Which is the most densely populated country in the world?

### Population Density in terms of area

In [54]:
### Population density vs area
database10 ="SELECT * FROM facts WHERE (population > (SELECT AVG(population) FROM facts)) AND (area < (SELECT AVG(area) FROM facts))"
pd.read_sql_query(database10,conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,14,bg,Bangladesh,148460,130170,18290,168957745,1.60,21.14,5.61,0.46
1,65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
2,85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.00
3,138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
4,173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.80,0.00
5,185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
6,192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.30


### Other Method to see Population Density

In [21]:
### population density vs area
database11 = "SELECT *, cast(population as float)/cast(area as float) density from facts order by density desc limit 20"
pd.read_sql_query(database11, conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,density
0,205,mc,Macau,28,28,0.0,592731,0.80,8.88,4.22,3.37,21168.964286
1,117,mn,Monaco,2,2,0.0,30535,0.12,6.65,9.24,3.83,15267.500000
2,156,sn,Singapore,697,687,10.0,5674472,1.89,8.27,3.43,14.05,8141.279770
3,204,hk,Hong Kong,1108,1073,35.0,7141106,0.38,9.23,7.07,1.68,6445.041516
4,251,gz,Gaza Strip,360,360,0.0,1869055,2.81,31.11,3.04,0.00,5191.819444
5,233,gi,Gibraltar,6,6,0.0,29258,0.24,14.08,8.37,3.28,4876.333333
6,13,ba,Bahrain,760,760,0.0,1346613,2.41,13.66,2.69,13.09,1771.859211
7,108,mv,Maldives,298,298,0.0,393253,0.08,15.75,3.89,12.68,1319.640940
8,110,mt,Malta,316,316,0.0,413965,0.31,10.18,9.09,1.98,1310.015823
9,227,bd,Bermuda,54,54,0.0,70196,0.50,11.33,8.23,1.88,1299.925926
